In [1]:
import json

from langchain.chains.question_answering import load_qa_chain

from langchain_community.llms import Ollama
from langchain_community.embeddings.laser import LaserEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatMaritalk

from langchain_core.prompts.chat import ChatPromptTemplate

from langchain.text_splitter import RecursiveCharacterTextSplitter

import numpy as np

import pandas as pd


In [25]:
# obtains the API key used to access Maritalk's API
with open('API_KEY.json', 'r') as file:
    maritalk = json.load(file)

model = ChatMaritalk(
    model='sabia-3',
    api_key=maritalk['key'],
    temperature=0.7,
    max_tokens=500,
)

In [67]:
# ollama_base_url= 'http://tempestade.facom.ufms.br:11435'
ollama_base_url = 'http://localhost:11434' 
model = Ollama(base_url=ollama_base_url, model="phi3:medium", temperature=0)

# ollama_base_url = 'http://localhost:11434' 
# model = Ollama(base_url=ollama_base_url, model="llama3.1, temperature=0)

# embeddings_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

In [5]:
embeddings_model = LaserEmbeddings(lang='por_Latn')

2025-03-22 16:59:45,904 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2025-03-22 16:59:46,120 | INFO | laser_encoders.download_models |  - laser2.spm already downloaded
2025-03-22 16:59:46,180 | INFO | laser_encoders.download_models |  - laser2.pt already downloaded
2025-03-22 16:59:46,181 | INFO | laser_encoders.download_models |  - laser2.spm already downloaded
2025-03-22 16:59:46,181 | INFO | laser_encoders.download_models |  - laser2.cvocab already downloaded


In [41]:
doc_folder = 'edital2'

In [40]:
documents_dir_path = f'./database/{doc_folder}'

loader = DirectoryLoader(documents_dir_path, glob='./*.pdf', loader_cls=PyPDFLoader)

loaded_pdfs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=0,
        separators=['\n', ' ']
)

pages = text_splitter.split_documents(loaded_pdfs)

In [13]:
vectorstore = FAISS.from_documents(
    pages,
    embeddings_model
)

vectorstore.save_local(f'database/{doc_folder}/document_index')

In [42]:
# loads vectorstore from disk
vectorstore = FAISS.load_local(f'database/{doc_folder}/document_index', embeddings_model, allow_dangerous_deserialization=True)

In [8]:
def assistant(content: str):
    return ("assistant", content)

def user(content: str):
    return ("user", content)

In [76]:
prompt_structure = '''
        Baseado nos seguintes documentos:
        {context}
        Responda a pergunta abaixo:
        {query}
        '''

In [ ]:
# role based prompt
qa_prompt = ChatPromptTemplate.from_messages([
        user('''Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.
         Você sempre cita o item do edital que contém a resposta desejada. Se não souber a resposta, responda "Não consigo encontrar essa informação no documento". 
         Você cita seomente item necessário para resposta direta da pergunta e nada mais.'''),
        ('human', prompt_structure)
])

In [ ]:
# few-shot prompting
qa_prompt = ChatPromptTemplate.from_messages([
        user('''Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.
         Você sempre cita o item do edital que contém a resposta desejada. Se não souber a resposta, responda "Não consigo encontrar essa informação no documento". 
         Você cita seomente item necessário para resposta direta da pergunta e nada mais. Aqui estão alguns exemplos:'''),
        user('''Como será a lista de espera?'''),
        assistant('''De acordo com o item 3.4 do edital, a lista de espera será definida pela ordem de cadastro aprovado 
          e permanecerá para o atendimento por meio da liberação de novas vagas pelo MEC.'''),
        user('''Qual o número mínimo de membros das comissões temporárias constituídas pelo Conselho?'''),
        assistant('''De acordo com o Art. 61, as comissões temporárias deverão ser constituídas por, no mínimo, três membros.'''),
        user(prompt_structure)
])

In [ ]:
chain = load_qa_chain(model, chain_type='stuff', verbose=True, prompt=qa_prompt)

In [68]:
query = input()

In [71]:
retrieval_result = vectorstore.similarity_search_with_relevance_scores(query, k=10, score_threshold=0.5)

In [70]:
# enriching the retrieval result 

# get the page index of the page with the most similar chunk
page_idx = retrieval_result[0][0].metadata['page']

retrieval_result = np.concatenate(
    (
        [x for x in pages if x.metadata['page'] == page_idx],
        [page[0] for page in retrieval_result]),
    axis=0
)

NameError: name 'pages' is not defined

In [72]:
# retrieved context, no enrichment
retrieval_result= [page[0] for page in retrieval_result]

In [73]:
output = chain.invoke(
        {'input_documents':retrieval_result, 'query':query}
)

print(output['output_text'])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.
         Você sempre cita o item do edital que contém a resposta desejada. Se não souber a resposta, responda "Não consigo encontrar essa informação no documento". 
         Você cita seomente item necessário para resposta direta da pergunta e nada mais. Aqui estão alguns exemplos:
Human: Como será a lista de espera?
AI: De acordo com o item 3.4 do edital, a lista de espera será definida pela ordem de cadastro aprovado 
          e permanecerá para o atendimento por meio da liberação de novas vagas pelo MEC.
Human: Qual o número mínimo de membros das comissões temporárias constituídas p

### Geração das respostas

In [58]:
qa_prompt.get_prompts()[0]

ChatPromptTemplate(input_variables=['context', 'query'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento\n         de todo o documento apresentado como contexto e \n         responde todas as perguntas em Portugues do Brasil. Você responde a perguntas sobre o documento apresentado, usando o contexto fornecido.\n         Você sempre cita o item do edital que contém a resposta desejada. Se não souber a resposta, responda "Não consigo encontrar essa informação no documento". \n         Você cita seomente item necessário para resposta direta da pergunta e nada mais. Aqui estão alguns exemplos:')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Como será a lista de espera?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='De acordo com o item 3.4 do edital, a lista de espera será definida pela ordem de

In [39]:
queries = pd.read_csv('tests/test_questions.csv', dtype={'numero':int})

enrichment = True

answers = []

for num_doc in [70, 9]:

    questions = queries[queries['numero'] == num_doc]['pergunta'].tolist()

    for question in questions:
        
        query = f'De acordo com o edital {num_doc}. ' + question

        retrieval_result = vectorstore.similarity_search_with_relevance_scores(query, k=10)

        if enrichment:
            # enriching the retrieval result 

            # get the page index of the page with the most similar chunk
            page_idx = retrieval_result[0][0].metadata['page']

            retrieval_result = np.concatenate(
                (
                    [x for x in pages if x.metadata['page'] == page_idx],
                    [page[0] for page in retrieval_result]),
                axis=0
            )
        else:
            # retrieved context, no enrichment
            retrieval_result= [page[0] for page in retrieval_result]

        output = chain.invoke(
            {'input_documents':retrieval_result, 'query':query}
        )

        answers.append(output['output_text'])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Você é um funcionário da Universidade Federal de Mato Grosso do Sul que tem conhecimento
         de todo o documento apresentado como contexto e 
         responde todas as perguntas em Portugues do Brasil.
Human: 
        Baseado nos seguintes documentos:
        e) Comprovante ou Certiﬁcado de Conclusão do Ensino
Médio em escola pública ou privada com bolsa de estudo integral.
 
6.2. O correto preenchimento da Ficha de Inscrição e a inclusão
dos documentos são de inteira responsabilidade do estudante.
 
6.3. A inscrição implicará no seu declarado conhecimento e
tácita aceitação das regras, exigências e condições estabelecidas neste
Edital.
 
7. DO PROCESSO SELETIVO, CLASSIFICAÇÃO, CRITÉRIOS
DE DESEMPATE, RESULTADO E RECURSO

7.1. Os inscritos serão classiﬁcados de acordo com distribuição
de vagas descrita no item 3.1 deste Edital.
 
7.2. Os critérios para classiﬁcação do

In [40]:
result = pd.DataFrame(
    {
        'pergunta':queries['pergunta'].tolist(),
        'resposta':answers,
        'edital':queries['arquivo'].tolist()
    }
)

In [41]:
result

,pergunta,resposta,edital
0,Quem pode receber a Bolsa Permanência do MEC?,"De acordo com o Edital, a Bolsa Permanência do...",edital-1
1,Qual o período de inscrições para a Bolsa Perm...,"De acordo com o Edital 70, o período de inscri...",edital-1
2,Quais documentos são necessários para a inscri...,"De acordo com o Edital nº 70, os estudantes de...",edital-1
3,Onde os estudantes devem se inscrever para a B...,"De acordo com o Edital, os estudantes devem se...",edital-1
4,Quais são os motivos para a suspensão da Bolsa...,"De acordo com o Edital nº 70, a suspensão da B...",edital-1
5,Quantas bolsas estão disponíveis para estudant...,O Edital não especifica um número exato de bol...,edital-1
6,Como é definida a lista de espera?,"De acordo com Edital 70 (DO PROCESSO SELETIVO,...",edital-1
7,Quando serão divulgados os resultados?,"De acordo com o Edital Nº 70, os resultados do...",edital-1
8,Qual é o objetivo do Edital PROAES/PROGRAD/PRO...,O objetivo do Edital PROAES/PROGRAD/PROPP/UFMS...,edital-2
9,Quais são os valores das bolsas oferecidas par...,"De acordo com o Edital, as bolsas oferecidas t...",edital-2


In [42]:
result.to_csv(f'tests/results/{model.model}-enrichment-vanilla-prompt.csv', index=False)